In [ ]:
## RAG 전체 구조

### 저장 (Indexing)
1. Document Load → 문서를 Document 객체로 변환
2. Split → 청크 단위로 분할
3. Embedding → 청크를 벡터로 변환
4. Vector Store → 벡터 DB에 저장

### 검색 + 생성 (Retrieval & Generation)
5. Retrieval → 사용자 질문을 벡터화 → 유사한 청크 검색
6. Prompt → 검색된 청크 + 질문을 프롬프트에 조합
7. LLM 생성 → 답변 생성

## 저장 파트

1. Document Load (PDF)
   file_path 설정 → loader 인스턴스 생성(객체 초기화를 위한 과정) → .load() 실행
   - loader 인스턴스 (file_path)
   📦 입력: PDF 파일 → 출력: Document 리스트 (페이지별)

2. Splitting
    text_splitter 인스턴스 생성(객체 초기화를 위한 과정) -> .split_documents() 실행
    -  text_splitter 인스턴스 (`chunk_size` , `chunk_overlap`)
    - .split_documents(`로드한 문서`)
   📦 입력: Document 리스트 → 출력: 작은 청크 리스트

3. Embedding (숫자로 바꾸기)
    `embeddings 인스턴스 생성 (임베딩 모델 준비만 해두는 단계)`
    embeddings 인스턴스 생성
    - embeddings 인스턴스 (model = `'모델'`)
    - `실제 변환은 4번에서 add_documents() 시 내부적으로 실행됨`
   📦 입력: 청크 텍스트 → 출력: 벡터(숫자 배열)


4. Vector Store 저장
    vecotr_store 인스턴스 생성 -> .add_documents() 실행 (`즉, vector_sotre 인스턴스에 청크들을 추가해주는 작업임`)
    - vecotr_store 인스턴스 (`임베딩 모델 객체`)
    - .add_documents(documents = `split한 document 데이터들`)
    - `임베딩 변환 + 저장이 한꺼번에 이루어짐`
   📦 입력: 벡터 + 청크 → 출력: 검색 가능한 DB


## 검색 파트

-> 벡터 저장소를 검색기로 변환
    vector_store (저장소)
        ↓ .as_retriever() (검색 설정 세팅)

    - .as_retriever(`search_type`,`search_kwargs`)
    - search_type (검색 기준)
    - search_kwargs (검색 기준으로 가장 높은 상위 n개 가져오기 )
    
## 실제 검색 
-> 위에서 만든 검색기를 기반으로 검색
    retriever (검색기)
    ↓ .invoke("질문")
    - .invoke(`질문사항`)
    - 질문 -> 벡터화 -> 유사도 비교

## 검색기를 Agent에 Tool(함수)로 달기 위해 함수 생성

- 벡터 저장소. 검색기준 (질문, 상위 몇개)  -> 리스트로 나옴

예를 들어 split 후 각 Document 객체의 metadata에 "start_index": 1200 같은 값이 들어가서, 그 청크가 원본 텍스트의 1200번째 문자부터 시작된다는 걸 알 수 있습니다.